# Import

In [1]:
import pandas as pd
from sodapy import Socrata
from api_keys import *
import datetime

In [2]:
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("www.dallasopendata.com", app_token)

In [3]:
#grab data from api
#sdr7-6v3j is the dataset code from sodapy
all_data = client.get_all("sdr7-6v3j")

#turn into pandas df
data_df = pd.DataFrame.from_records(all_data)

In [4]:
data_df.sort_values(by='ararrestdate',ascending=False, inplace=True)
# data_df

In [5]:
# data_df.columns
#looking for only certain columns
#'incidentnum', 'arrestnumber', 'ararrestdate', 'ararresttime', 'arpremises', 'arladdress', 'arlzip','sex','drugrelated','drugtype','age'
data_df = data_df[['incidentnum', 'arrestnumber', 'ararrestdate', 'ararresttime', 'arpremises', 'arladdress', 'arlzip','sex','drugrelated','drugtype','age']]
# data_df.head()

In [6]:
data_df = data_df.loc[((data_df['drugrelated']=='Yes') | (data_df['drugrelated']=='Uknown'))]
# data_df.head()

In [8]:
for index, row in data_df.iterrows():
        split = row['ararrestdate'].split('T')
        row['ararrestdate'] = split[0]


In [9]:
data_df.head()

,incidentnum,arrestnumber,ararrestdate,ararresttime,arpremises,arladdress,arlzip,sex,drugrelated,drugtype,age
39,198577-2022,22-031360,2022-11-01,23:00,"Highway, Street, Alley ETC",9600 C F HAWN SERV WB,75217,Male,Yes,Methamphetamine,42
50,198023-2022,22-031292,2022-11-01,05:40,NaN,8550 N STEMMONS SERV NB,75247,Male,Yes,Cultivated Marijuana,22
166,198195-2022,22-031311,2022-11-01,13:00,Convenience Store,2603 S LANCASTER RD,75216,Male,Yes,Cultivated Marijuana,23
58,198025-2022,22-031285,2022-11-01,05:40,"Highway, Street, Alley ETC",2700 N STEMMONS FWY,75207,Male,Uknown,NaN,23
85,198496-2022,22-031347,2022-11-01,20:19,Outdoor Area Public/Private,4503 S LANCASTER RD,75216,Male,Uknown,NaN,66


In [10]:
import sqlite3

In [11]:
conn =sqlite3.connect('test_database')
c = conn.cursor()

In [12]:
c.execute('CREATE TABLE IF NOT EXISTS crime_test (incidentnum, arrestnumber, ararrestdate, ararresttime, arpremises, arladdress, arlzip, sex, drugrelated, drugtype, age)')
conn.commit()

In [14]:
data_df.to_sql('crime_test', conn, if_exists='replace', index = False)

20525

In [15]:
c.execute('''select * from crime_test''')

In [17]:
for row in c.fetchall():
    print(row)
    break

In [23]:
#decrease the amount of data to filter by today or past week, month, and year
#creating the variables to filter through the df's
today = datetime.date.today()
week = today - datetime.timedelta(days=7)
month = today - datetime.timedelta(days=30)
year = (today - datetime.timedelta(days=365)).strftime('%Y-%m-%d')
today = today.strftime('%Y-%m-%d')
week = week.strftime('%Y-%m-%d')
month = month.strftime('%Y-%m-%d')

In [24]:
#filter through the dataframes descending to not constantly work with big data for each filter
# year_df = data_df[(data_df['ararrestdate'] >= year)]
# month_df = year_df[(year_df['ararrestdate'] >= month)]
# week_df = month_df[(month_df['ararrestdate'] >= week)]
# today_df = week_df[(week_df['ararrestdate'] >= today)]

# filters = [year_df, month_df, week_df, today_df]